In [1]:
'''
made by weiyw @ 2019-4-10
GAN
'''
## network workflow
import os
import keras
import segyio
import datetime
import numpy as np
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D,Conv2DTranspose
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Lambda,
    Reshape,
    Dropout,
    Concatenate
)
os.environ["CUDA_VISIBLE_DEVICES"]="3"

Using TensorFlow backend.


In [2]:
class GANDataloader():
    def __init__(self, data_path, nt, nr, nph):
        self.data_path = data_path
        self.nt = nt
        self.nr = nr
        self.nph = nph


    def normaliza(self, datain):
        for batch_ii in range( datain.shape[0] ):
            for iterm_ii in range( datain.shape[-1] ):
                target = datain[batch_ii, :, :, iterm_ii]
                mind = target.min() #datain[batch_ii, :, :, iterm_ii].min()
                maxd = target.max() #datain[batch_ii, :, :, iterm_ii].max()      
                for ii in range(target.shape[0]):
                    for jj in range(target.shape[1]):
                        if target[ii, jj] > 0: 
                            target[ii,jj] = target[ii,jj] / maxd
                        if target[ii, jj] < 0:
                            target[ii,jj] = target[ii,jj] / ( 0 - mind )
                datain[batch_ii, :, :, iterm_ii] = target
    #     return datain
    
    def load_batch(self, batch_size=1, is_testing=False, ratio=0.5):        
        self.n_batches = int( 151 / batch_size * ratio ) #int( len(path) / batch_size * ratio )
        x_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-2001-467-4, acc
        y_data = np.empty((batch_size, self.nt, self.nr, 1)) ## b-2001-467-1, div, curl
#         i = 0
        with segyio.open(self.data_path,'r',ignore_geometry=True) as segyfile: 
            segyfile.mmap()
#             while True:
            for i in range(self.n_batches):
#                 if (i + 1) * batch_size > 151 * ratio:
#                     i = 0
#                     break
                for batch_i in range(batch_size):
                    for nr_i in range(self.nr):
#                     with segyio.open(self.data_path,'r',ignore_geometry=True) as segyfile:      
                        y_data[batch_i,:,nr_i,0] = \
                        segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 0]
                        x_data[batch_i,:,nr_i,0] = \
                        segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 1]
                        x_data[batch_i,:,nr_i,1] = \
                        segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 2]
                        x_data[batch_i,:,nr_i,2] = \
                        segyfile.trace[i*batch_size*4*self.nr + batch_i * 4 * self.nr + nr_i * 4 + 3]                 
                self.normaliza(x_data)
                self.normaliza(y_data)
                yield x_data, y_data
#                 i = i + 1


In [3]:
class vspGAN():
    def __init__(self):
        self.gf = 64
        self.df = 64
        self.nt = 2001
        self.nr = 467
        self.nph = 3
        self.nop = 1
        
        self.disc_patch = (int(self.nt / 2**4)+1, int(self.nr / 2**4)+1, 1)
        optimizer = Adam(0.0002, 0.5)
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
        
        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        data_B = Input( shape=( self.nt, self.nr, self.nph) )
        data_A = Input( shape=( self.nt, self.nr, self.nop) )

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(data_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([data_B, fake_A])

        self.combined = Model(inputs=[data_B, data_A], outputs=[fake_A,valid])
        self.combined.compile(loss=['mse', 'mae'],loss_weights=[1, 100],optimizer=optimizer) 
        
    def build_generator(self):  
        '''
        U-net generator
        '''
        def cutLayer(xx, target):
            return xx[:, 0:int(target.shape[1]),0:int(target.shape[2]),0:int(target.shape[3])]
    
        def conv2d(layer_input, filters, f_size=(4,4), s_size=(2,2), bn=True):
            """Layers used during downsampling"""
            xx = Conv2D(filters=filters, kernel_size=f_size, strides=s_size, padding='same', 
                        data_format='channels_last', dilation_rate=(1, 1), activation=None, 
                        use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros', 
                        kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                        kernel_constraint=None, bias_constraint=None)(layer_input)
            if bn:
                xx = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                           beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
                           moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                           beta_constraint=None, gamma_constraint=None)(xx)
            return xx
        

        def deconv2d(layer_input, skip_input, filters, f_size=(4,4),s_size=(2,2), dropout_rate=0, if_skip=True,
                     if_last=False):
            """Layers used during upsampling"""
            xx = Conv2DTranspose(filters=filters, kernel_size=f_size, strides=s_size, padding='same', output_padding=None, 
                         data_format='channels_last', dilation_rate=(1, 1), activation=None, use_bias=True, 
                         kernel_initializer='glorot_uniform', bias_initializer='zeros', 
                         kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                         kernel_constraint=None, bias_constraint=None)(layer_input)
            if if_last:
                xx = Conv2DTranspose(filters=filters, kernel_size=f_size, strides=s_size, padding='same', output_padding=None, 
                             data_format='channels_last', dilation_rate=(1, 1), activation='tanh', use_bias=True, 
                             kernel_initializer='glorot_uniform', bias_initializer='zeros', 
                             kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
                             kernel_constraint=None, bias_constraint=None)(layer_input)
                
            if if_skip and xx.shape != skip_input.shape:
                xx = Lambda(cutLayer, arguments={'target':(skip_input)})(xx)#(xx,skip_input)
                
            if dropout_rate:   
                xx = Dropout(dropout_rate)(xx)
                
            if not if_last:
                xx = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                           beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
                           moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                           beta_constraint=None, gamma_constraint=None)(xx)
                xx = Concatenate()([xx, skip_input])
            return xx

        # Image input
        d0 = Input( shape=( self.nt, self.nr, self.nph) )#shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*8)
        d3 = conv2d(d2, self.gf*16)
        d4 = conv2d(d2, self.gf*16)

        # Upsampling
    
        u1 = deconv2d(d4, d3, self.gf*16)
        u2 = deconv2d(u1, d2, self.gf*16)
        u3 = deconv2d(u2, d1, self.gf*8)
        u4 = deconv2d(u3, d0, self.nop, if_last=True)

        return Model(inputs = d0, outputs = u4)
    
    
    def build_discriminator(self):  
        '''
        U-net discriminator
        ''' 
        def d_layer(layer_input, filters, f_size=(4,4),s_size=(2,2), bn=True):
            
            d = Conv2D(filters, kernel_size=f_size, strides=s_size, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d
        data_B = Input( shape=( self.nt, self.nr, self.nph) )
        data_A = Input( shape=( self.nt, self.nr, self.nop) )
        
        # Concatenate image and conditioning image by channels to produce input
        combined_data = Concatenate(axis=-1)([data_B, data_A])

        d1 = d_layer(combined_data, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([data_B, data_A], validity)
        

In [4]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [5]:
epochs = 3; batch_size = 3; sample_interval = 2; ratio=0.5

data_path = '/home/wyw/data/SEAM_I_walkaway_vsp_s23900/SEAM_Well1VSP_Shots23900.sgy'
save_dir = os.path.join(os.getcwd(), 'saved_models')
out_name = 'test_GAN'
nt = 2001; nr = 467; ns = 151; nph = 3; nop = 1

In [6]:
# opt = parameters().parse()
GANmodel = vspGAN()

D = GANmodel.discriminator
G = GANmodel.generator
C = GANmodel.combined
# my_data_loader = GANDataloader(opt.data_path, opt.nt, opt.nr, opt.nph)
my_data_loader = GANDataloader(data_path, nt, nr, nph)

# filepath = os.path.join(save_dir,'{}.best.h5'.format(out_name))
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=0, save_best_only=True)

In [7]:
# G.summary()

In [8]:
# C.summary()

In [ ]:
# D.summary()

In [ ]:
disc_patch = GANmodel.disc_patch
# disc_patch = (126,30,1)
valid = np.ones((batch_size,) + disc_patch )
fake = np.zeros((batch_size,) + disc_patch ) 
print("milestones00")
start_time = datetime.datetime.now()
print("milestones0")
for epoch in range(epochs):
    for batch_i, (data_B, data_A) in enumerate(
        my_data_loader.load_batch(batch_size=batch_size, is_testing=False, ratio=ratio)):
        # ---------------------
        #  Train Discriminator
        # ---------------------
        print("milestones1")
        # Condition on B and generate a translated version
        fake_A = G.predict(data_B)
        
        # Train the discriminators (original images = real / generated = Fake)
#         D.trainable = True#False
        d_loss_real = D.train_on_batch([data_B, data_A], valid)
        d_loss_fake = D.train_on_batch([data_B, fake_A], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        print("milestones2")
        # -----------------
        #  Train Generator
        # -----------------

        # Train the generators
        g_loss = C.train_on_batch([data_B, data_A], [data_A, valid])
        print("milestones3")
        elapsed_time = datetime.datetime.now() - start_time
        # Plot the progress
        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % 
               (epoch+1, epochs, batch_i+1, my_data_loader.n_batches,
                d_loss[0], 100*d_loss[1], g_loss[0], elapsed_time))

        # If at save interval => save generated image samples
#         if batch_i % sample_interval == 0:
#             self.sample_data(epoch, batch_i)
    if epoch % sample_interval == 0:
        this_name = out_name + "-" + str(epoch+1)
        D.save_weights(os.path.join( save_dir, '{}-D.h5'.format(this_name)), overwrite=True)
        G.save_weights(os.path.join( save_dir, '{}-C.h5'.format(this_name)), overwrite=True)
        print("model saved")
#             C.save_weights(os.path.join( save_dir, '{}-C-final-best.h5'.format(out_name)), overwrite=True)
print("training finished")

milestones00
milestones0
milestones1


/home/wyw/anaconda3/envs/tf-keras/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


milestones2
milestones3
[Epoch 1/3] [Batch 1/25] [D loss: 24.313932, acc:  23%] [G loss: 125.235130] time: 0:00:46.062967
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 2/25] [D loss: 10.006067, acc:  12%] [G loss: 253.147614] time: 0:01:03.458822
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 3/25] [D loss: 2.598898, acc:  20%] [G loss: 202.376373] time: 0:01:20.953811
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 4/25] [D loss: 5.896624, acc:  13%] [G loss: 151.195312] time: 0:01:38.392088
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 5/25] [D loss: 2.363771, acc:  33%] [G loss: 151.043213] time: 0:01:55.823097
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 6/25] [D loss: 5.116420, acc:  13%] [G loss: 256.153717] time: 0:02:13.201690
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 7/25] [D loss: 1.751547, acc:  21%] [G loss: 111.867828] time: 0:02:30.580753
milestones1
milestones2
milestones3
[Epoch 1/3] [Batch 8/25] [D loss: 2.045421,